SCRAPE SUPREME COURT 2016 ORAL ARGUMENTS PDF AND MAKE A DATAFRAME BASED ON SPEAKER. 
MERGE TWO DATASETS BASED ON DOCKET NAME.

In [1]:
import requests
from bs4 import BeautifulSoup
response = requests.get('https://www.supremecourt.gov/oral_arguments/argument_transcript.aspx')
doc = BeautifulSoup(response.text, 'html.parser')

ConnectionError: HTTPSConnectionPool(host='www.supremecourt.gov', port=443): Max retries exceeded with url: /oral_arguments/argument_transcript.aspx (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10dfc4be0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [37]:
table = doc.find(class_="table datatables")
cases = table.find_all('tr')
cases

NameError: name 'doc' is not defined

In [ ]:
#List of all of the PDFs, names, docket names, date argued.
#Using a for loop after determining the whole scraped table
all_2016 = []
for things in cases:
    current = {}
    case_id = things.find_all('td')[0].find('a')
    name = things.find('span')
    date = things.find_all('td')[1].string
    docket_number = things.find_all('td')[0].find(target="_blank")
    
    if name:
        current['Case Name'] = name.string
        
    if case_id:
        this_file = case_id['href'].split('/')[-1]
        current['case_id'] = this_file.split('.')[0]
        
        current['Date Argued'] = date
        current['Docket Number'] = docket_number.string.strip()
        all_2016.append(current)
all_2016

In [2]:
#Now that we have a list of dicionaries, create dataframes with pandas 
import pandas as pd
df = pd.DataFrame(all_2016)
df.shape

NameError: name 'all_2016' is not defined

In [ ]:
df.columns

In [ ]:
df['Docket Number'] = df['Docket Number'].str.replace(".","")

In [ ]:
df.head()

In [ ]:
array = df['case_id'].unique()
array.sort()
array

In [ ]:
#Import the regular expression library
import re
!pwd

In [3]:
#Open a text file from your computer
f = open('pdfs/14-1055_h3dj.txt', 'r')
sample_transcript = f.read()

In [4]:
#Take a look at the text file
sample_transcript

'Official - Subject to Final Review\n1 1 IN THE SUPREME COURT OF THE UNITED STATES\n\n2 -----------------x\n\n3 CRYSTAL MONIQUE\n\n:\n\n4 LIGHTFOOT, ET AL.,\n\n:\n\n5\n\nPetitioners\n\n: No. 14-1055\n\n6 v.\n\n:\n\n7 CENDANT MORTGAGE\n\n:\n\n8 CORPORATION, DBA PHH\n\n:\n\n9 MORTGAGE, ET AL.,\n\n:\n\n10\n\nRespondents.\n\n:\n\n11 - - - - - - - - - - - - - - - - - x\n\n12 Washington, D.C.\n\n13 Tuesday, November 8, 2016\n\n14\n\n15 The above-entitled matter came on for oral\n\n16 argument before the Supreme Court of the United States\n\n17 at 11:04 a.m.\n\n18 APPEARANCES:\n\n19 E. JOSHUA ROSENKRANZ, ESQ., New York, N.Y.; on behalf of\n\n20 the Petitioners.\n\n21 ANN O\'CONNELL, ESQ., Assistant to the Solicitor General,\n\n22 Department of Justice, Washington, D.C.; for United\n\n23 States, as amicus curiae, supporting the Petitioners.\n\n24 BRIAN P. BROOKS, ESQ., Washington, D.C.; on behalf of\n\n25 the Respondents.\n\nAlderson Reporting Company\n\n\x0cOfficial - Subject to Final Review\

In [5]:
def parse_transcript(the_text):
    remove_alderson = re.sub(r"Alderson Reporting Company", "", the_text, flags = re.IGNORECASE)
    remove_official = re.sub(r'Official - Subject to Final Review', "", remove_alderson, flags = re.IGNORECASE)
    remove_numbers = re.sub(r"\b([1-9]|1[0-9]|2[0-5])\b", " ", remove_official) 
    new_split = re.split(r"PROCEEDINGS", remove_numbers)
    new_clean = new_split[1]
    edit_bottom = re.split(r"above-entitled", new_clean)
    isolated_middle = edit_bottom[0]
    edit_n = re.sub(r"\n", " ", isolated_middle)
    edit_x = re.sub(r"\x0c", "", edit_n) 
    replace1 = re.sub(r"\w+ ARGUMENT[^a-z]+THE PETITIONERS", "", edit_x)
    replace2 = re.sub(r"\w+ ARGUMENT[^a-z]+THE PETITIONER", "", replace1)
    replace3 = re.sub(r"\w+ ARGUMENT[^a-z]+THE RESPONDENTS", "", replace2)
    replace4 = re.sub(r"\w+ ARGUMENT[^a-z]+ (PETITIONER|RESPONDENT)S?", "", replace3)
    speakers = re.split(r"([A-Z][A-Z.\s]+):", replace4)
    del speakers[:3]
    full_list = list(zip(speakers[0::2], speakers[1::2]))
    return full_list

In [6]:
parse_transcript(sample_transcript)

NameError: name 're' is not defined

In [7]:
#looping through several cases
list_of_cases = []
path = 'pdfs/'
for file_name in array:
    print(file_name)
    if file_name != '15-1358_7648' and file_name != '15-577_l64n' and file_name != '15-866_j426' and file_name != '16-32_mlho' and file_name!= '16-466_4g15' and file_name !='16-529_21p3':
        f = open(path + file_name + '.txt', 'r')
        sample_transcript = f.read()
        this_list = parse_transcript(sample_transcript)   
        better_list = []
        for each in this_list:
            entry = list(each)
            entry.append(file_name)
            better_list.append(entry)
        this_list.append(file_name)
        list_of_cases.extend(better_list)

NameError: name 'array' is not defined

In [8]:
import pandas as pd
col_names = ['Speaker','Speech', 'PDF']
df2 = pd.DataFrame.from_records(list_of_cases, columns=col_names)
df2.head()

,Speaker,Speech,PDF


MERGING THE TWO DATAFRAMES 

In [9]:
combined_df = df2.merge(df, left_on='PDF', right_on='case_id')
combined_df.shape

NameError: name 'df' is not defined

In [10]:
combined_df['Speaker'].value_counts()


NameError: name 'combined_df' is not defined

In [11]:
combined_df.shape

NameError: name 'combined_df' is not defined

In [12]:
from collections import Counter
import re
#https://docs.python.org/2/library/collections.html
wordcount = combined_df.groupby(["Speaker"]).sum().applymap(lambda words: Counter(re.findall(r"\b\w{6,}\b",words.lower())).most_common())

NameError: name 'combined_df' is not defined

In [13]:
wordcount

NameError: name 'wordcount' is not defined

In [14]:
len(combined_df['Speaker'])

NameError: name 'combined_df' is not defined

In [15]:
combined_df.groupby('Speaker')['Speech'].nunique().sort_values(ascending=True)

NameError: name 'combined_df' is not defined

In [16]:
#combined_df['Speaker'].str.contains('ROBERTS').value_counts()
#combined_df['Speaker'].str.contains('ALITO').value_counts()
#combined_df['Speaker'].str.contains('THOMAS').value_counts()
#combined_df['Speaker'].str.contains('KENNEDY').value_counts()
#combined_df['Speaker'].str.contains('KAGAN').value_counts()
#combined_df['Speaker'].str.contains('ROBERTS').value_counts()
#combined_df['Speaker'].str.contains('SOTOMAYOR').value_counts()
#combined_df['Speaker'].str.contains('JUSTICE').value_counts()
#combined_df['Speaker'].str.contains('MR').value_counts()
#combined_df['Speaker'].str.contains('MS').value_counts()

In [17]:
scotus_df = pd.read_csv('Updated Supreme Court State Information - Sheet1.csv')
scotus_df

,Docket Number,State,Date Argued,Date Decided,Area*,Decision,Status,Court Leaning,Previous Court,City,Latitude,Longitude
0,14-1055,California,06/28/16,01/18/17,Economic Activity,8-0,affirmed,Major Majority,9th Circuit,"Pasadena, California",34.147785,-118.144516
1,14-1538,Wisconsin,06/27/16,02/22/17,Economic Activity,7-0,reversed,Major Majority,District Court for the Western District of Wis...,"Madison, Wisconsin",43.073052,-89.401230
2,14-9496,Illinois,01/15/16,03/21/17,Civil Rights,6-2,affirmed,Major Majority,7th Circuit,"Joilet, Illinois",41.525031,-88.081725
3,15-118,Texas,10/11/16,06/26/17,Immigration,NaN,vacated,Vacated,5th Circuit,"San Antonio, Texas",29.424122,-98.493628
4,15-214,Wisconsin,01/15/16,06/23/17,Economic Activity,5-3,affirmed,Liberal,Wisconsin Court Of Appeals District III,"St. Croix County, Wisconsin",45.049784,-92.387569
5,15-423,"Washington, D.C.",06/28/16,05/01/17,Economic Activity,8-0,reversed,Major Majority,United States District Court for the District ...,"Washington, D.C.",38.907192,-77.036871
6,15-457,California,01/15/16,06/12/17,Economic Activity,8-0,reversed,Major Majority,9th Circuit,"Seattle, Washington",47.606209,-122.332071
7,15-497,Michigan,06/28/16,02/22/17,Civil Rights,8-0,affirmed,Major Majority,6th Circuit,"Detriot, Michigan",42.331427,-83.045754
8,15-513,Louisiana,11/01/16,Pending,Economic Activity,8-0,reversed,Major Majority,5th Circuit,"Gulf Port, Mississippi",30.367420,-89.092816
9,15-537,Puerto Rico,10/04/16,11/29/16,Criminal Procedure,8-0,reversed,Major Majority,1st Circuit,Puerto Rico,18.220833,-66.590149


In [18]:
scotus_df['State'].value_counts().head()

California          10
Washington, D.C.     9
Texas                6
Arizona              3
Florida              3
Name: State, dtype: int64

In [19]:
scotus_df['Area*'].value_counts()

Economic Activity       19
Criminal Procedure      12
Civil Rights             9
Immigration              4
Economic Activity        4
Due Process              4
First Amendment          3
Judicial Power           2
Civil Rights             2
Employment               1
Private Action           1
Crime                    1
Miscellaneous            1
Interstate Relations     1
Name: Area*, dtype: int64

In [20]:
scotus_df['Court Leaning'].value_counts()

Major Majority    43
Liberal            8
Conservative       6
can not tell       2
Mixed              2
Pending            2
Vacated            1
Name: Court Leaning, dtype: int64

MERGING DF AND SCOTUS_DF DATAFRAMES TOGETHER 


In [21]:
full_df = pd.merge(df, scotus_df, on='Docket Number')
full_df.head()

NameError: name 'df' is not defined

ANOTHER MERGED - ON 2ND AND 3RD DATAFRAME

In [22]:
df_speech = pd.merge(combined_df, scotus_df, on='Docket Number')
df_speech

NameError: name 'combined_df' is not defined

In [23]:
df_civil = df_speech['Area*'] == 'Civil Rights'
df_civil.value_counts()

NameError: name 'df_speech' is not defined

In [24]:
df_speech['Area*'].value_counts()

NameError: name 'df_speech' is not defined

ISOLATING CIVIL RIGHTS CASES

In [25]:
df_speech.groupby('Area*')['Speaker'].value_counts() 

NameError: name 'df_speech' is not defined

In [26]:
df_civil = df_speech.loc[df_speech['Area*'] == 'Civil Rights']
df_civil


NameError: name 'df_speech' is not defined

COUNT NUMBER OF LINES PER CASE PER SPEAKER

In [27]:
df_new = df_civil.groupby('Speaker')['Case Name'].value_counts().sort_values(ascending=False).reset_index(name='count')
df_new

NameError: name 'df_civil' is not defined

In [28]:
#sort by case number
df_new.sort_values("Case Name")

NameError: name 'df_new' is not defined

In [29]:
#Number of speakers per Civil Rights Case 
#df_new['Total # of Speakers'] = 
df_new['Case Name'].value_counts()

NameError: name 'df_new' is not defined

In [30]:
df_civil.head()

NameError: name 'df_civil' is not defined

In [31]:
#lenght of lines of each Civil Rights Case 
df_civil['Docket Number'].value_counts()

NameError: name 'df_civil' is not defined

In [32]:
#most frequent words used in each civil rights case 
import re
from collections import Counter
wordcount_case = df_civil.groupby(["Case Name"]).sum().applymap(lambda words: Counter(re.findall(r"\b\w{6,}\b",words.lower())).most_common())

NameError: name 'df_civil' is not defined

In [33]:
wordcount_case

NameError: name 'wordcount_case' is not defined

In [34]:
df_another = wordcount_case[['Case Name', 'Speech', 'Docket Number']]
df_another
                            

NameError: name 'wordcount_case' is not defined

In [35]:
#make a new column with readable speech
#lamba function 
#wordcount['Speech']

In [36]:
#combined_df = df2.merge(df, left_on='PDF', right_on='case_id')
#df_speech = pd.merge(combined_df, scotus_df, on='Docket Number')

In [ ]:
#df_final = pd.merge(wordcount_case, df_new, on='Docket Number')

In [ ]:
df_new.groupby('Case Name')

In [ ]:
#Number of lines per case 
#df_final = pd.merge(wordcount_case, df_new, on='Docket Number')
new_full = df_new.groupby(["Case Name"]).sum()

In [ ]:
new_full.reset_index(inplace = True)
new_full

In [ ]:
first_merge = pd.merge(new_full, df_new, on = 'Case Name')
first_merge

In [ ]:
new_merge = pd.merge(new_full, full_df, on='Case Name')
new_merge

In [ ]:
new_merge.drop(new_merge.columns[[4, 5, 6, 7, 8, 12]], axis=1, inplace = True)

In [ ]:
new_merge.rename(columns={'count': '# Of Lines', 'Date Argued_x': 'Date'}, inplace=True)


In [ ]:
new_merge

In [ ]:
def unpack(freq_words):
    string = ""
    for pair in freq_words[0:5]:
        string += pair[0] + ": " + str(pair[1]) + " "
    return string

In [ ]:
wordcount_case['WordsString'] = wordcount_case['Speech'].apply(lambda x: unpack(x)) 

In [ ]:
wordcount_cases

In [ ]:
wordcount_case.drop(wordcount_case.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13]], axis=1, inplace = True)

In [ ]:
wordcount_case

In [ ]:
wordcount_case.rename(columns={'WordsString': 'Most Frequent Word'}, inplace=True)
wordcount_case

In [ ]:
df_withmore = pd.merge(wordcount_case, new_merge, on='Case Name')

In [ ]:
df_withmore

In [ ]:
#now make sure to get the most frequent speaker 

In [ ]:
def unpack(freq_words):
    string = ""
    for pair in freq_words[0:5]:
        string += pair[0] + ": " + str(pair[1]) + " "
    return string

In [ ]:
wordcount_case['Frequent Speaker'] = wordcount_case['Speaker'].apply(lambda x: unpack(x)) 